<a href="https://colab.research.google.com/github/NavaneethRaj17/image-generator/blob/init/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Input,Dense,Reshape,Flatten
from keras.layers import BatchNormalization,LeakyReLU
from keras.models import Sequential,Model
from keras.optimizers import Adam

import ssl
import urllib.request

ssl._create_default_https_context = ssl._create_unverified_context
(X_train,_),(_,_) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [2]:
# Generator Network
def build_generator():
  model = Sequential() #layers will be added one after the other in a linear stack.

  #First Layer
  model.add(Dense(256,input_dim=100)) #fully connected (dense) layer with 256 neurons. input_dim is input to this layer will be a 100-dimensional noise vector
  model.add(LeakyReLU(alpha=0.2)) ## Leaky ReLU activation function, which allows a small, non-zero gradient when the input is negative (helps to prevent "dying ReLU" issues).
  model.add(BatchNormalization(momentum=0.8)) #Normalizes the output of the previous layer

  #Second Layer
  model.add(Dense(1024)) #dense layer with 1024 neurons, further increasing the model's capacity
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  #Output Layer

  #layer outputs 784 values (since 28x28 = 784)
  model.add(Dense(784,activation='tanh')) #tanh activation function scales the output to the range [-1, 1], which is suitable for image data when pixel values are normalized.
  model.add(Reshape((28,28,1))) #Reshapes the output into a 3D tensor with dimensions (28, 28, 1), corresponding to a grayscale image of size 28x28.
  return model

generator = build_generator()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [3]:
# Discriminator Network - a binary classifier that outputs the probability of an image being real

def build_descriminator():
  model = Sequential()
  model.add(Flatten(input_shape=(28,28,1))) #flattens the input image (28x28 pixels, 1 channel) into a 1D array of 784 elements (28 * 28).
  #This prepares the data for the fully connected layers that follow

  #First Layer
  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))

  #Second Layer
  model.add(Dense(256))
  model.add(LeakyReLU(alpha=0.2))

  #Output Layer
  # sigmoid activation function is used here to squash the output into this range.
  model.add(Dense(1,activation='sigmoid')) #single neuron that outputs a value between 0 and 1, representing the probability that the input image is real
  return model

discriminator = build_descriminator()
discriminator.compile(optimizer=Adam(0.0002,0.5),loss="binary_crossentropy",metrics=['accuracy'])
#BCE can be interpreted as the likelihood of the model's predictions. It effectively measures how well the predicted probabilities align with the actual class labels.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [4]:
# Compiling and Training
# The discriminator will be trained to distinguish real from fake images while the generator will be trained to produce images that fool the discriminator



discriminator.trainable = False

gan_input = Input(shape=(100,))
generated_image = generator(gan_input)
gan_output = discriminator(generated_image)

gan = Model(gan_input,gan_output)
gan.compile(optimizer=Adam(0.0002,0.5),loss='binary_crossentropy')

def train_gan(epochs,batch_size=64):
  X_train,_ = mnist.load_data()
  X_train = (X_train[0].astype(np.float32) - 127.5) / 127.5
  X_train = np.expand_dims(X_train,axis=3)

  real = np.ones((batch_size,1))
  fake = np.zeros((batch_size,1))

  for epoch in range(epochs):
    idx = np.random.randint(0,X_train.shape[0],batch_size)
    real_images = X_train[idx]

    noise = np.random.normal(0,1,(batch_size,100))
    generated_images = generator.predict(noise)

    d_loss_real = discriminator.train_on_batch(real_images,real)
    d_loss_fake = discriminator.train_on_batch(generated_images,fake)
    d_loss = 0.5 * np.add(d_loss_real,d_loss_fake)

    noise = np.random.normal(0,1,(batch_size,100))
    g_loss = gan.train_on_batch(noise,real)

    if epoch % 100 == 0:
      print(f"{epoch} [D loss: {d_loss[0]},acc. : {100*d_loss[1]}] [G loss: {g_loss}]")
      save_images(epoch)

def save_images(epoch):
  r,c=5,5
  noise = np.random.normal(0,1,(r*c,100))
  generated_images = generator.predict(noise)

  generator_images = 0.5*generated_images + 0.5

  fig,axs = plt.subplots(r,c)
  count = 0
  for i in range(r):
    for j in range(c):
      axs[i,j].imshow(generated_images[count,:,:,0],cmap='gray')
      axs[i,j].axis('off')
      count +=1
  fig.savefig(f"gan_images_{epoch}.png")
  plt.close()

train_gan(epochs=1000,batch_size=32)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


0 [D loss: 0.6021112203598022,acc. : 80.46875] [G loss: [array(0.6194631, dtype=float32), array(0.6194631, dtype=float32), array(0.796875, dtype=float32)]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━